In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lib.rl.trajectory import serve_trajectory_batch_api

server = serve_trajectory_batch_api(None)

In [4]:
import os

os.getpid()

11847

In [ ]:
import requests

requests.get("http://localhost:8000/")

In [ ]:
from lib.rl.trajectory import TrajectoryBatchRequest
import requests

response = requests.post(
    "http://localhost:8000/trajectory-batch",
    json=TrajectoryBatchRequest(
        dir="./tensors", rows=1_000, seqlen=8192, start=0, stop=10
    ).model_dump(mode="json"),
    timeout=5,
)
response

In [6]:
from IPython.display import HTML
from lib.rl import Completion, Episode
from lib.tokenizer import Tokenizer

tokenizer = Tokenizer("NousResearch/Hermes-2-Theta-Llama-3-8B")
completion = Completion.model_validate_json(open("best_trajectory.json").read())
episode = Episode([], lambda _: None)
episode.completion = completion
terminus = episode.best_leaf(tokenizer=tokenizer)

INFO 11-08 20:01:28 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='NousResearch/Hermes-2-Theta-Llama-3-8B', speculative_config=None, tokenizer='NousResearch/Hermes-2-Theta-Llama-3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=NousResearch/Hermes-2-Theta-Llama-3-8B, use_v2_block_manager=True, num_scheduler_steps=1, chunked_

In [29]:
HTML(f'<div style="white-space: pre-wrap">{terminus.html()}</div>')

In [24]:
for leaf in completion.leaves():
    if leaf.value() == 1.0:
        display(HTML(f'<div style="white-space: pre-wrap">{leaf.html()}</div>'))
        break

In [27]:
leaf.advantage()

0.2944809993108114